## Setup

In [1]:
import os,math,sys,torch,re,numpy as np
from types import SimpleNamespace as ns

In [2]:
np.set_printoptions(precision=2, linewidth=140)
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)

In [3]:
sys.path.insert(0, '..')

In [4]:
from utils import show_img,load_cuda,cuda_begin

In [5]:
%load_ext wurlitzer

In [6]:
os.environ['CUDA_LAUNCH_BLOCKING']='1'
torch.manual_seed(42);

In [7]:
m1 = torch.rand(50_000, 784)
m1s = m1[:8]
m2 = torch.rand(784,10)

## Reminder

### 2d Python kernel

In [8]:
def blk_kernel2d(f, blocks, threads, *args):
    for i0 in range(blocks.y):
        for i1 in range(blocks.x):
            for j0 in range(threads.y):
                for j1 in range(threads.x): f(ns(x=i1,y=i0), ns(x=j1,y=j0), threads, *args)

In [9]:
def matmul_bk(blockidx, threadidx, blockdim, m, n, out, h, w, k):
    r = blockidx.y*blockdim.y + threadidx.y
    c = blockidx.x*blockdim.x + threadidx.x
    
    if (r>=h or c>=w): return
    o = 0.
    for i in range(k): o += m[r*k+i] * n[i*w+c]
    out[r*w+c] = o

In [10]:
def matmul_2d(m, n):
    h,k  = m.shape
    k2,w = n.shape
    assert k==k2, "Size mismatch!"
    output = torch.zeros(h, w, dtype=m.dtype)
    tpb = ns(x=16,y=16)
    blocks = ns(x=math.ceil(w/tpb.x), y=math.ceil(h/tpb.y))
    blk_kernel2d(matmul_bk, blocks, tpb,
                 m.flatten(), n.flatten(), output.flatten(), h, w, k)
    return output

In [11]:
torch.isclose(matmul_2d(m1s, m2), m1s@m2).all()

tensor(True)

### CUDA

In [12]:
cuda_src = cuda_begin + r'''
__global__ void matmul_k(float* m, float* n, float* out, int h, int w, int k) {
    int r = blockIdx.y*blockDim.y + threadIdx.y;
    int c = blockIdx.x*blockDim.x + threadIdx.x;

    if (r>=h || c>=w) return;
    float o = 0;
    for (int i = 0; i<k; ++i) o += m[r*k+i] * n[i*w+c];
    out[r*w+c] = o;
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k==n.size(0), "Size mismatch!");
    auto output = torch::zeros({h, w}, m.options());

    dim3 tpb(16,16);
    dim3 blocks(cdiv(w, tpb.x), cdiv(h, tpb.y));
    matmul_k<<<blocks, tpb>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

In [13]:
fname = 'matmul'

In [14]:
def get_sig(fname, src):
    res = re.findall(rf'^(.+\s+{fname}\(.*?\))\s*{{?\s*$', src, re.MULTILINE)
    return res[0]+';' if res else None

In [15]:
cpp_src = get_sig(fname, cuda_src)
cpp_src

'torch::Tensor matmul(torch::Tensor m, torch::Tensor n);'

In [16]:
module = load_cuda(cuda_src, cpp_src, [fname])

In [17]:
m1c,m2c = m1.contiguous().cuda(),m2.contiguous().cuda()

In [18]:
module.matmul(m1c,m2c).shape

torch.Size([50000, 10])

In [19]:
torch.isclose(module.matmul(m1c,m2c), m1c@m2c).all()

tensor(True, device='cuda:0')

## Shared mem

### 2d Python kernel

In [20]:
a = torch.zeros(5)
b,c = a[:3],a[3:]

In [21]:
b[1] = 2
c[0] = 6
a

tensor([0., 2., 0., 6., 0.])

In [22]:
def blk_kernel2d_shar(f, blocks, threads, sh_sz, *args, **kwargs):
    for i0 in range(blocks.y):
        for i1 in range(blocks.x):
            shared = torch.zeros(sh_sz)
            f(ns(x=i1,y=i0), threads, shared, *args, **kwargs)

In [23]:
def run_threads(f, blockdim, *args, **kwargs):
    for i0 in range(blockdim.y):
        for i1 in range(blockdim.x): f(i0, i1, *args, **kwargs)

In [24]:
def matmul_tiled_bk(blockidx, blockdim, shared, m, n, out, h, w, k, tw):
    shar_sz = tw*tw
    ms,ns = shared[:shar_sz],shared[shar_sz:]

    def get_rc(ty, tx): return blockidx.y*blockdim.y + ty, blockidx.x*blockdim.x + tx

    def fill_shared_tk(ty, tx, ph):
        r,c = get_rc(ty, tx)
        ms[ty*tw+tx] = m[ tx + ph*tw + r*k] if r<h and (ph*tw+tx)<k else 0.
        ns[ty*tw+tx] = n[(ty + ph*tw)*w +c] if c<w and (ph*tw+ty)<k else 0.

    def dotprod_tk(ty, tx):
        r,c = get_rc(ty, tx)
        for i in range(tw):
            if r*w+c<len(out): out[r*w+c] += ms[ty*tw+i] * ns[tw*i+tx]

    for ph in range(int(math.ceil(k/tw))):
        run_threads(fill_shared_tk, blockdim, ph)
        run_threads(dotprod_tk, blockdim)

In [25]:
def matmul_2d(m, n, tw=16):
    h,k  = m.shape
    k2,w = n.shape
    assert k==k2, "Size mismatch!"
    output = torch.zeros(h, w, dtype=m.dtype)
    tpb = ns(x=tw,y=tw)
    blocks = ns(x=math.ceil(w/tpb.x), y=math.ceil(h/tpb.y))
    blk_kernel2d_shar(matmul_tiled_bk, blocks, tpb, tw*tw*2,
                      m.flatten(), n.flatten(), output.flatten(),
                      h, w, k, tw=tw)
    return output

In [26]:
m1s.shape, m2.shape

(torch.Size([8, 784]), torch.Size([784, 10]))

In [27]:
torch.isclose(matmul_2d(m1s, m2, tw=16), m1s@m2).all()

tensor(True)

### 2d Python kernel threads

In [28]:
import threading
from threading import Barrier, Thread
from concurrent.futures import ThreadPoolExecutor

In [29]:
def g(x, sb):
    print(x)
    sb.wait()
    print(-x)
    sb.wait()
    print(x*10)

In [30]:
num = 3
sb = Barrier(num)
with ThreadPoolExecutor(num) as ex: list(ex.map(lambda i: g(i,sb), range(num)))

0
1
2
-2
-1
0
10
0
20


In [31]:
def blk_kernel2d_shar(f, blocks, tpb, sh_sz, *args, **kwargs):
    for i0 in range(blocks.y):
        for i1 in range(blocks.x):
            shar = torch.zeros(sh_sz)
            syncb = Barrier(tpb.y*tpb.x)
            threads = [Thread(target=f, args=(ns(x=i1,y=i0), ns(x=p,y=o), tpb, shar, syncb, *args), kwargs=kwargs)
                       for o in range(tpb.y) for p in range(tpb.x)]
            for t in threads: t.start()
            for t in threads: t.join()

In [32]:
def matmul_tiled_bk(blockidx, threadidx, blockdim, shared, syncb, m, n, out, h, w, k, tw):
    tx,ty = threadidx.x,threadidx.y
    r = blockidx.y*blockdim.y + ty
    c = blockidx.x*blockdim.x + tx

    shar_sz = tw*tw
    ms,ns = shared[:shar_sz],shared[shar_sz:]

    p = 0.
    for ph in range(int(math.ceil(k/tw))):
        ms[ty*tw+tx] = m[ tx + ph*tw + r*k] if r<h and (ph*tw+tx)<k else 0.
        ns[ty*tw+tx] = n[(ty + ph*tw)*w +c] if c<w and (ph*tw+ty)<k else 0.
        syncb.wait()
        for i in range(tw): p += ms[ty*tw+i] * ns[tw*i+tx]
        syncb.wait()

    if (r<h and c<w): out[r*w + c] = p

In [33]:
def matmul_2d(m, n, tw=16):
    h,k  = m.shape
    k2,w = n.shape
    assert k==k2, "Size mismatch!"
    output = torch.zeros(h, w, dtype=m.dtype)
    tpb = ns(x=tw,y=tw)
    blocks = ns(x=math.ceil(w/tpb.x), y=math.ceil(h/tpb.y))
    blk_kernel2d_shar(matmul_tiled_bk, blocks, tpb, tw*tw*2,
                      m.flatten(), n.flatten(), output.flatten(),
                      h, w, k, tw=tw)
    return output

In [34]:
%%time
torch.isclose(matmul_2d(m1s, m2, tw=8), m1s@m2).all()

CPU times: user 8.8 s, sys: 10.4 s, total: 19.2 s
Wall time: 10.1 s


tensor(True)

### CUDA

#### Manual Tiling

In [15]:
#include <cuda_runtime.h>
#include <math.h>
cuda_src = cuda_begin + r'''
#define TW 16
__global__ void matmul_k(float *m, float *n, float *out, int h, int w, int k) {
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int bw = blockDim.x;
    int bh = blockDim.y;

    int row = by * bh + ty;
    int col = bx * bw + tx;

    __shared__ float ms[TW*TW];
    __shared__ float ns[TW*TW];

    float p = 0.0;

    for (int ph = 0; ph < ceil((float)k / TW); ++ph) {
        int mIndex = row * k + ph * TW + tx;
        int nIndex = (ph * TW + ty) * w + col;
        
        if (row < h && (ph * TW + tx) < k)
            ms[ty * TW + tx] = m[mIndex];
        else
            ms[ty * TW + tx] = 0.0;

        if (col < w && (ph * TW + ty) < k)
            ns[ty * TW + tx] = n[nIndex];
        else
            ns[ty * TW + tx] = 0.0;

        __syncthreads();

        for (int i = 0; i < TW; ++i)
            p += ms[ty * TW + i] * ns[i * TW + tx];

        __syncthreads();
    }

    if (row < h && col < w)
        out[row * w + col] = p;
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k==n.size(0), "Size mismatch!");
    auto output = torch::zeros({h, w}, m.options());

    dim3 tpb(16,16);
    dim3 blocks(cdiv(w, tpb.x), cdiv(h, tpb.y));

    printf("blocks.x: %u blocks.y: %u", blocks.x, blocks.y);
    printf("tpb.x: %u tpb.y: %u", tpb.x, tpb.y);


    matmul_k<<<blocks,tpb>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

In [16]:
fname = 'matmul'

In [17]:
def get_sig(fname, src):
    res = re.findall(rf'^(.+\s+{fname}\(.*?\))\s*{{?\s*$', src, re.MULTILINE)
    return res[0]+';' if res else None

In [18]:
cpp_src = get_sig(fname, cuda_src)
cpp_src

'torch::Tensor matmul(torch::Tensor m, torch::Tensor n);'

In [19]:
module = load_cuda(cuda_src, cpp_src, [fname], verbose=True)

Using /home/ubuntu/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
The input conditions for extension module inline_ext have changed. Bumping to version 1 and re-building as inline_ext_v1...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu121/inline_ext/build.ninja...
Building extension module inline_ext_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=inline_ext_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/include -isystem /home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/include/TH -isystem /home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /home/ubuntu/miniconda3/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/ubuntu/.cache/torch_extensions/py311_cu121/inline_ext/main.cpp -o main.o 
[2/3] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output cuda.cuda.o.d -DTORCH_EXTENSION_NAME=inline_ext_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUIL

Loading extension module inline_ext_v1...


In [20]:
m1c,m2c = m1.contiguous().cuda(),m2.contiguous().cuda()

In [21]:
module.matmul(m1c,m2c).shape

torch.Size([50000, 10])

blocks.x: 1 blocks.y: 3125tpb.x: 16 tpb.y: 16

In [22]:
torch.isclose(module.matmul(m1c,m2c), m1c@m2c).all()

blocks.x: 1 blocks.y: 3125tpb.x: 16 tpb.y: 16

tensor(True, device='cuda:0')

#### Dynamic Tiling

Here we `extern __shared__ float ms_ns[]` to dynamically allocate the shared memory, which might vary based on the device properties.

In [26]:
#include <cuda_runtime.h>
#include <math.h>
cuda_src = cuda_begin + r'''
__global__ void matmul_k(float *m, float *n, float *out, int h, int w, int k, int TW, unsigned size) {
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int bw = blockDim.x;
    int bh = blockDim.y;

    int row = by * bh + ty;
    int col = bx * bw + tx;

    extern __shared__ float ms_ns[];
    float *ms = ms_ns;
    float *ns = &ms_ns[TW*TW]; // Use number of elements to offset

    // printf("ms_size: %u, ns_size: %u", size/2, size/2);

    float p = 0.0;

    for (int ph = 0; ph < ceil((float)k / TW); ++ph) {
        int mIndex = row * k + ph * TW + tx;
        int nIndex = (ph * TW + ty) * w + col;
        
        if (row < h && (ph * TW + tx) < k)
            ms[ty * TW + tx] = m[mIndex];
        else
            ms[ty * TW + tx] = 0.0;

        if (col < w && (ph * TW + ty) < k)
            ns[ty * TW + tx] = n[nIndex];
        else
            ns[ty * TW + tx] = 0.0;

        __syncthreads();

        for (int i = 0; i < TW; ++i)
            p += ms[ty * TW + i] * ns[i * TW + tx];

        __syncthreads();
    }

    if (row < h && col < w)
        out[row * w + col] = p;
}

size_t calculateSharedMemorySize() {
    // Query the first CUDA device
    cudaDeviceProp devProp;
    cudaError_t cudaStatus = cudaGetDeviceProperties(&devProp, 0);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "cudaGetDeviceProperties failed: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 0; // Return 0 if unable to get device properties
    }

    // Use maxThreadsPerBlock from device properties
    int maxThreads = devProp.maxThreadsPerBlock;

    // Calculate the required size for shared memory based on maxThreads
    size_t requiredSize = static_cast<size_t>(maxThreads) * 2 * 4; // Assuming 4 bytes per element (float)

    // Take the minimum of devProp.sharedMemPerBlock and requiredSize
    size_t size = min(devProp.sharedMemPerBlock, requiredSize);

    // Output the calculated size
    std::cout << "Calculated shared memory size: " << size << " bytes" << std::endl;

    return size;
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k==n.size(0), "Size mismatch!");
    auto output = torch::zeros({h, w}, m.options());

    size_t size = calculateSharedMemorySize();
    // size_t size = 2048; // test with fixed size
    
    printf("Maximum shared memory per block: %zu bytes\n", size);

    int TW = std::sqrt(size / 2 / sizeof(float));
    dim3 tpb(TW,TW);
    dim3 blocks(cdiv(w, tpb.x), cdiv(h, tpb.y));

    printf("TW: %u ", TW);
    printf("blocks.x: %u blocks.y: %u\n", blocks.x, blocks.y);
    printf("tpb.x: %u tpb.y: %u\n", tpb.x, tpb.y);

    matmul_k<<<blocks,tpb,size>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k, TW, size);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

In [27]:
fname = 'matmul'

In [28]:
def get_sig(fname, src):
    res = re.findall(rf'^(.+\s+{fname}\(.*?\))\s*{{?\s*$', src, re.MULTILINE)
    return res[0]+';' if res else None

In [29]:
cpp_src = get_sig(fname, cuda_src)
cpp_src

'torch::Tensor matmul(torch::Tensor m, torch::Tensor n);'

In [30]:
module = load_cuda(cuda_src, cpp_src, [fname], verbose=True)

Using /home/ubuntu/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
The input conditions for extension module inline_ext have changed. Bumping to version 3 and re-building as inline_ext_v3...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu121/inline_ext/build.ninja...
Building extension module inline_ext_v3...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=inline_ext_v3 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/include -isystem /home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/include/TH -isystem /home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /home/ubuntu/miniconda3/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/ubuntu/.cache/torch_extensions/py311_cu121/inline_ext/main.cpp -o main.o 
[2/3] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output cuda.cuda.o.d -DTORCH_EXTENSION_NAME=inline_ext_v3 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUIL

Loading extension module inline_ext_v3...


In [31]:
m1c,m2c = m1.contiguous().cuda(),m2.contiguous().cuda()

In [32]:
module.matmul(m1c,m2c).shape

Calculated shared memory size: 8192 bytes
Maximum shared memory per block: 8192 bytes
TW: 32 blocks.x: 1 blocks.y: 1563
tpb.x: 32 tpb.y: 32


torch.Size([50000, 10])

In [33]:
torch.isclose(module.matmul(m1c,m2c), m1c@m2c).all()

Calculated shared memory size: 8192 bytes
Maximum shared memory per block: 8192 bytes
TW: 32 blocks.x: 1 blocks.y: 1563
tpb.x: 32 tpb.y: 32


tensor(True, device='cuda:0')